In [ ]:
%matplotlib inline
import calendar
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine

# pretty plots
import seaborn as sns
# import matplotlib
# matplotlib.style.use('ggplot')

In [ ]:
DATABASE_URL = "sqlite:///msgstore.db"

In [ ]:
engine = create_engine(DATABASE_URL)
df = pd.read_sql('SELECT key_remote_jid, timestamp, key_from_me FROM messages '
                 'WHERE key_remote_jid LIKE "%@s.whatsapp.net"', engine)

# convert timestamp to datetime object
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('timestamp')

df['year'] = df.index.map(lambda x: x.year)
df['month'] = df.index.map(lambda x: x.month)

print df.head()

In [ ]:
# monthly chat traffic analysis
month_df = pd.DataFrame({
    'overall': df.groupby('month').month.count(),
    '2013': df[df['year'] == 2013].groupby('month').month.count(),
    '2014': df[df['year'] == 2014].groupby('month').month.count(),
    '2015': df[df['year'] == 2015].groupby('month').month.count(),
})
month_df.plot(kind='area', stacked=False, figsize=(16, 9), 
              grid=True, x_compat=True)

In [ ]:
# (in)frequently contacted friends
df.groupby('key_remote_jid')\
  .key_remote_jid\
  .count()\
  .order(ascending=False)
  .plot(figsize=(16, 9), logy=True)

In [ ]:
# speed of sending messages
# sent messages
tx_df = pd.DataFrame({
    'sent': df[df['key_from_me'] == 1].key_from_me,
})

# received messages
rx_df = pd.DataFrame({
    'received': df[df['key_from_me'] == 0].key_from_me,
})
rx_df['received'] = 1  # important, since key_from_me = 0
# print tx_df.describe(), rx_df.describe()

# full outer join rx and tx
rxtx_df = rx_df.merge(tx_df, left_index=True, right_index=True, how='outer')
rxtx_df = rxtx_df.fillna(0)
rxtx_df['received'] = rxtx_df['received'].map(lambda x: 1 if x > 0 else 0)
rxtx_df['sent'] = rxtx_df['sent'].map(lambda x: 1 if x > 0 else 0)
rxtx_df['total'] = rxtx_df['received'] + rxtx_df['sent']

rxtx_df.cumsum().plot(figsize=(16, 9))

In [ ]:
# day of week analysis
import time
def day_of_week(date_time):
    return "{}{}".format(date_time.weekday(),
                         calendar.day_name[date_time.weekday()])


weekday_df = total_df.groupby(day_of_week, axis=0).count()
weekday_df = weekday_df.merge(rx_df.groupby(day_of_week, axis=0).count(),
                              left_index=True, right_index=True)
weekday_df = weekday_df.merge(tx_df.groupby(day_of_week, axis=0).count(),
                              left_index=True, right_index=True)

weekday_df.plot(figsize=(16, 9))
print weekday_df.T